In [ ]:
import pandas as pd
import numpy as np
import cv2
import os
from tensorflow.keras.models import load_model
from sklearn.metrics import classification_report, accuracy_score, precision_score, recall_score, f1_score

# Cargar el modelo
model = load_model('chest_xray_model.h5')

# Leer el conjunto de prueba
test_df = pd.read_csv('divided_data/test.csv')

# Definir las etiquetas de enfermedades
disease_labels = test_df.columns[2:]  # Saltar 'Image' y 'PatientId'

# Crear una función para cargar y preprocesar las imágenes
def load_image(img_path, target_size=(224, 224)):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, target_size)
    img = img / 255.0
    img = np.expand_dims(img, axis=-1)
    return img

# Realizar las predicciones para el conjunto de prueba
y_true = []
y_pred = []

for _, row in test_df.iterrows():
    img_path = os.path.join('path_to_images_dir', 'images', row['Image'])
    img = load_image(img_path)
    img = np.expand_dims(img, axis=0)  # Agregar dimensión batch
    prediction = model.predict(img)[0]

    y_true.append(row[disease_labels].values.astype('float32'))
    y_pred.append(prediction)

y_true = np.array(y_true)
y_pred = np.array(y_pred)


In [ ]:
# Calcular las métricas de evaluación
accuracy = accuracy_score(y_true.round(), y_pred.round())
precision = precision_score(y_true.round(), y_pred.round(), average='macro')
recall = recall_score(y_true.round(), y_pred.round(), average='macro')
f1 = f1_score(y_true.round(), y_pred.round(), average='macro')

print(f'Accuracy: {accuracy:.4f}')
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')

# Generar un informe de clasificación detallado
report = classification_report(y_true.round(), y_pred.round(), target_names=disease_labels)
print(report)
